In [1]:
import gzip

with gzip.open("goodreads_books.json.gz",'r') as f:
    line= f.readline()

In [2]:
import json
data = json.loads(line)
data

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [17]:
def parse_fields(line):
    data = json.loads(line)
    return {
        "book_id": data["book_id"], 
        "title": data["title_without_series"],
        "pages": data["num_pages"],
        "ratings": data["ratings_count"], 
        "url": data["url"], 
        "cover_image": data["image_url"]
    }

In [41]:
books_titles = []
with gzip.open("goodreads_books.json.gz") as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_fields(line)
        try:
            ratings = int(fields["ratings"])
            pages = int(fields["pages"])
        except ValueError:
            continue
        if ratings > 5:
            books_titles.append(fields)

In [42]:
import pandas as pd

titles = pd.DataFrame.from_dict(books_titles)

In [43]:
titles["ratings"] = pd.to_numeric(titles["ratings"])

In [44]:
titles["pages"] = pd.to_numeric(titles["pages"])

In [45]:
titles["mod_title"] = titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

In [46]:
titles["mod_title"] = titles["mod_title"].str.lower()

In [47]:
titles["mod_title"] = titles["mod_title"].str.replace("\s+", " ", regex=True)

In [48]:
titles = titles[titles["mod_title"].str.len() > 0]

In [49]:
titles.to_json("books_titles.json")

In [50]:
titles

,book_id,title,pages,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",600,140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,368,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,162,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,216,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,147,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
...,...,...,...,...,...,...,...
1246299,30244126,Herança,336,6,https://www.goodreads.com/book/show/30244126-h...,https://images.gr-assets.com/books/1463811788m...,herana
1246300,3084038,"This Sceptred Isle, Vol. 10: The Age of Victor...",3,12,https://www.goodreads.com/book/show/3084038-th...,https://images.gr-assets.com/books/1494763458m...,this sceptred isle vol 10 the age of victoria ...
1246301,26168430,Sherlock Holmes and the July Crisis,148,6,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-assets.com/books/1440592011m...,sherlock holmes and the july crisis
1246302,2342551,The Children's Classic Poetry Collection,96,36,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,the childrens classic poetry collection


In [55]:
titles = titles[titles["pages"]<300]

In [65]:
book_recs = titles.sort_values("ratings", ascending=False)

AttributeError: 'int' object has no attribute 'sort_values'

In [64]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

book_recs.style.format({'url': make_clickable, 'cover_image': show_image})   

AttributeError: 'int' object has no attribute 'style'